# Теория вероятностей и математическая статистика
## Метод максимального правдоподобия
### Содержание

0. [Введение: выборка](#par0)
1. [Общая идея: что такое правдоподобие и функция правдоподобия](#par1)
2. [Пример MLE для биномиального распределения](#par2)
3. [Пример MLE для распределения Пуассона](#par3)
4. [Пример MLE для экспоненциального распределения](#par4)
5. [Пример MLE для нормального распределения](#par5)

In [ ]:
# !pip install numpy matplotlib seaborn scipy -q

### 0. Введение: выборка и предпосылки для понимания MLE <a name="par0"></a>

В этом конспекте-практикуме мы подробно поговорим об одном из самых популярных в статистике методе для оценивания неизвестного параметра — *методе максимального правдоподобия* (ММП / maximum likelihood estimation / MLE). Но, прежде всего, нужно освежить некоторые базовые понятия, имеющие прямое отношение к этому занятию: в частности, выборку и её статистические свойства.

#### Выборка

Хотя мы неоднократно употребляли термин **«выборка»** на занятиях, стоит четко концептуализировать это понятие, поскольку оно потребуется нам дальше.

- **Определение**: *выборка* — совокупность независимых и одинаково распределенных (independent and identically distributed / $i.i.d.$) случайных величин. При этом важно, что речь идет не только об одинаковом характере распределения сл. в., но и об одинаковых параметрах этого распределения (например, $N(0, 1)$ — среднее значение 0 и дисперсия 1 у нормального распределения).
- **Обозначение**: выборка обычно обозначается как $ X = (x_1, x_2, ..., x_n) $, где каждый $ x_i $ — это отдельное наблюдение, а $ n $ — размер выборки.

##### Ключевые концепции, связанные с выборками:

1. **Случайная выборка**: процесс формирования выборки таким образом, чтобы каждый элемент генеральной совокупности (ГС / population) имел равные шансы попасть в нее. Это критически важно для репрезентативности выборки.
2. **Размер выборки ($ n $)**: количество наблюдений в выборке. Оно играет значительную роль в статистическом выводе (statistical inference), влияя на основные свойства оценок (о них ниже).
3. **Параметр против оценки**: параметр ГС — это значение, описывающее характеристику всей популяции (и мы никогда не можем определить его в явном виде), в то время как оценка — это значение, описывающее неизвестный параметр распределения случайной величины, которое мы находим исходя из выборки.
Например, параметр «среднее значение»: $ \mu $ и выборочное среднее значение (оценка): $ \bar{x} $. По значению второго мы делаем предположение о первом. Часто параметр, в общем случае, обозначают как $ \theta $, а оценку — как $ \hat{\theta} $.
4. **Распределение выборки**: важно понимать распределение вероятностей случайной величины, из которой к нам приходит выборка. Мы уже изучили многие из них: нормальное, биномиальное, Пуассона, экспоненциальное и т.д. Каждое из этих распределений имеет свои параметры:
    * $ n $ (число независимых случайных экспериментов) и $ p $ (вероятность «успеха» в каждом из них) у биномиального распределения,
    * $ \lambda $ (среднее количество событий за фиксированный промежуток времени) у распределения Пуассона,
    * $ \lambda $ (интенсивность или частота) у экспоненциального распределения,
    * $ \mu $ и $ \sigma^2 $ (среднее и дисперсия) у нормального распределения,
    * и т.д.
    
Однако в реальных данных мы не всегда можем определить конкретное распределение вероятностей, из которого к нам пришла выборка.

#### Статистический вывод (statistical inference)

Статистический вывод — это фундаментальный аспект статистики, который включает в себя выводы о генеральной совокупности на основе выборки, взятой из этой ГС. Он охватывает различные методы и техники, используемые для оценки параметров, проверки гипотез и прогнозирования. Основная идея статистического вывода заключается в том, чтобы делать выводы о свойствах неизвестной популяции через меньшую, наблюдаемую выборку.

К примеру, в политической науке статистический вывод может использоваться для:
- анализа опросных данных (например, предпочтений избирателей, рейтингов одобрения и поддержки деятельности политиков),
- изучения эффектов реформ (например, влияния нового закона на общественное мнение),
- исследования взаимосвязей между переменными (например, демократические институты и экономическое развитие),
- прогнозирования результатов выборов.

Статистический вывод можно условно разделить на две большие части:
1. Оценивание параметров.
2. Проверка гипотез.

Частично мы уже затрагивали оба пункта, но сейчас мы поговорим именно про оценивание параметров, которое тоже можно разделить на две части:
1. Точечное оценивание.
2. Интервальное.

На 1 курсе Вы познакомились с интервальным оцениванием, когда строили доверительные интервалы для среднего значения и выборочной доли. Сейчас же мы подробнее остановимся на **точечном оценивании**, одним из самых популярных представителей которого и является метод максимального правдоподобия (однако стоит отметить, что на основе оценок MLE тоже можно построить доверительные интервалы)

### 1. Общая идея: что такое правдоподобие и функция правдоподобия <a name="par1"></a>

#### Правдоподобие

- **Определение**: правдоподобие — это концепт, используемый для измерения вероятности наблюдения данной выборки при различных значениях параметров.
- **Формальное определение**: для имеющейся выборки из наблюдаемых данных $ X = (x_1, x_2, ..., x_n) $ правдоподобие (*likelihood*) определяется как вероятность получения данной конкретной выборки (при заданном значении параметра $ \theta $): $ L(x_1, x_2, ..., x_n) = P(X) = P(X = x_1 \cap X = x_2 \cap ... \cap X = x_n) = P(X = x_1) \cdot P(X = x_2) \cdot ... \cdot P(X = x_n)$. Также правдоподобие можно записать как $ P(X | \theta) = P(\{data\} | \theta)$.
- **Интерпретация**: дословно, правдоподобие можно трактовать как ответ на следующий вопрос:
    1. «Если бы определенное значение параметра было верным, насколько вероятно было бы наблюдать имеющиеся у нас данные?»
    2. «Какая вероятность получить ту выборку, которую мы получили?»

К примеру, у нас есть «нечестная мотетка»: мы предполагаем, что вероятность выпадения «орла» составляет 0.8: $ \theta = 0.8$. Какое правдоподобие будет у следующей выборки: $\ \{ р, р, р, о, р \} $? Ответ: $ 0.2^4 \cdot 0.8 \approx 0.001 $. А у такой:$\ \{ о, о, о, о, о \} $? Ответ: $ 0.8^5 \approx 0.328 $ — гораздо более вероятно.

#### Функция правдоподобия

- **Определение**: функция правдоподобия — более формальное представление концепции правдоподобия. Данные из выборки подставляются в функцию правдоподобия для нахождения значений параметров, которые максимизируют эту функцию. По сути, мы используем данные из выборки для наилучшей возможной оценки параметров ГС, используя функцию правдоподобия.
- **Формальное определение**: функция правдоподобия является совместным распределение выборки из параметрического распределения, которое можно рассматривать как функцию параметра $ \theta $: $ L(\theta) = P(\theta | X) = (\theta | x_1, x_2, ..., x_n) = P(\theta | \{data\}) $. То есть, наоборот, мы хотим получить вероятность получения значения параметра $ \theta $ при условии выборки (данных).
- **Непрерывный случай**: для непрерывных распределений $ L(\theta) = f(\theta | x_1, x_2, ..., x_n) $, где $ f $ — совместная функция плотности вероятности.
- **Дискретный случай**: для дискретных распределений $ L(\theta) = P(\theta | x_1, x_2, ..., x_n) $, где $ P $ — совместная функция вероятности.

Неформально можно сказать, что мы используем правдоподобие, чтобы получить вероятность выборки при известных параметрах, а функцию правдоподобия — чтобы оценить вероятность неизвестных параметров при известных результатах (выборке).

Если более четко проследить разницу между ними, то получится, что:
- правдоподобие — это всегда числовое значение для данного параметра, которое можно посчитать с учетом выборки, а функция правдоподобия — функция, которая отражает эту взаимосвязь для всех возможных значений параметров,
- правдоподобие говорит нам о том, насколько вероятно конкретное значение параметра для наблюдаемых данных. В отличие от этого, функция правдоподобия используется как инструмент для оценки, являясь основой для оценки правдоподобия различных значений параметров.

---


#### Нахождение оценки параметра с использованием MLE

- **Предположения**: для применения MLE необходимо иметь представление о выборке и распределении случайной величины, из которой она пришла.

Процесс нахождения оценки максимального правдоподобия параметра $ \theta $ обычно включает следующие шаги:

1. **Построение функции правдоподобия**: на основе выбранного распределения и наблюдаемых данных нужно выписать функцию правдоподобия $ L(\theta | X) $.

2. **Логарифм функции правдоподобия (логарифмическое правдоподобие)**: для упрощения расчетов обычно от функции правдоподобия берут натуральный логарифм, получая функцию логарифмического правдоподобия $ \ln(L(\theta | X)) $.
    * Это делается для того, что бы упростить задачу нахождения параметра, который максимизирует функцию правдоподобия (т.е. точку эстремума или значение, при котором функция правдоподобия будет максимальной). Поскольку логарифм является монотонно возрастающей функцией, максимизация логарифма функции правдоподобия эквивалентна максимизации самой функции правдоподобия.
    * При этом, логарифм произведения является суммой логарифмов, что упрощает дифференцирование и позволяет легко избавиться от ненужных констант.
3. **Дифференцирование**: берем частную производную функции логарифмического правдоподобия по $ \theta $, чтобы получить $ \frac{d\ln(L(\theta | X))}{d\theta} $.

4. **Нахождение точки экстремума**: приравниваем производную к нулю и решаем уравнение относительно $ \theta $. Этот шаг включает нахождение значения $ \theta $, при котором функция логарифмического правдоподобия (а следовательно, и сама функция правдоподобия) достигает своего максимума. Таким образом, решение уравнения дает нам MLE-оценку $ \hat{\theta}^{MLE} $ параметра $ \theta $, указывая на точку экстремума, в которой функция правдоподобия является максимальной.
 Из этой логики берется темин «максимальное правдоподобие».

Важно отметить, что на практике (в частности, в статистических пакетах и языках программирования) часто используются численные методы и алгоритмы оптимизации для нахождения MLE, поскольку аналитическое решение зачастую очень трудоемко.

#### Свойства MLE-оценок

Стоит отметить *свойства* MLE-оценок в целом:

- **Асимптотическая несмещенность**: с увеличением размера выборки математическое ожидание оценки стремится к оцениваемому параметру, или $ \lim_{n \to \infty} E(\hat{\theta}_n) = \theta $. Это свойство указывает на то, что при больших объемах выборки оценка становится несмещенной.

- **Состоятельность**: оценка сходится по вероятности к истинному значению параметра по мере увеличения размера выборки. Это означает, что с увеличением объема выборки оценка становится более точной, или, формально, для любого $ \epsilon > 0 : \ \lim_{n \to \infty} P(|\hat{\theta}_n - \theta| < \epsilon) = 1 $.

- **Асимптотическая эффективность**: с увеличением размера выборки дисперсия MLE-оценок стремится к минимально возможной среди всех оценок. Это минимальное значение дисперсии оценок определяется через нижнюю границу, известную как неравенство (граница) Крамера-Рао, которая устанавливает теоретический минимум дисперсии, который может иметь несмещенная оценка параметра на основе данной выборки. Для MLE-оценки $ \hat{\theta} $ это означает, что при больших объемах выборки ее дисперсия $ Var(\hat{\theta}) $ приближается к этому минимуму, обеспечивая наивысшую точность оценки параметра среди всех возможных несмещенных оценок. Например, между двумя несмещенными оценками $ \hat{\theta}_1 $ и $ \hat{\theta}_2 $, $ \hat{\theta}_1 $ более эффективна, чем $ \hat{\theta}_2 $, если $Var(\hat{\theta}_1) < Var(\hat{\theta}_2) $

- **Чувствительность к выбросам (неустойчивость)**: подобно многим другим оценкам, MLE может быть чувствительной к выбросам в данных, что может повлиять на ее свойства, особенно на малых выборках. В таком случае MLE может дать смещенный результат.

---

### 2. Пример MLE для биномиального распределения <a name="par2"></a>

Рассмотрим два примера для биномиального распределения: найдем оценку параметра $p$ (вероятность «успеха») с помощью MLE, а также оценим вероятность получения выборок при известном параметре $p$ с помощью функции правдоподобия.

#### Поиск оценки параметра $p$

Представим, что был проведен референдум для оценки степени общественной поддержки некоторой политической реформы.
Мы имеем случайную выборку из 10 человек, которых опросили, поддерживают ли они реформу.

Предположим, что ответы следующие (1 означает поддержку, а 0 — отсутствие поддержки):
$$
X = \{1, 0, 1, 1, 0, 1, 1, 1, 0, 1\}
$$

В этой выборке $ n = 10 $ (общее количество наблюдений) и количество «успехов» составляет $ x = 7 $.

Биномиальное распределение подходит нам, поскольку оно описывает количество успехов в фиксированном количестве независимых испытаний Бернулли.

#### Функция правдоподобия для биномиального распределения

Функция правдоподобия для биномиального распределения, учитывая $ n $ испытаний и $ x $ успехов, имеет вид:
$$
L(p | X) = P(X = x | p) = C_n^x \ p^x (1 - p)^{n - x}
$$

Где:
- $L(p | X)$ — вероятность наблюдения $ x $ успехов при вероятности успеха $ p $.
- $C_n^x$ — биномиальный коэффициент.

#### Maximum likelihood estimation

Будем следовать алгоритму, который обсуждался выше, чтобы найти значение $ \theta $ (в нашем случае это  $ p $), которое максимизирует $ L(\theta | X) $. В начале найдем решение в общем виде, а затем подставим конкретные значения из условия.

1. **Логарифмическая функция правдоподобия**: сначала мы берем натуральный логарифм функции правдоподобия, чтобы упростить расчеты:

   $$
   \ln(L(p | X)) = \ln C_n^x + x \log p + (n - x) \log (1 - p)
   $$

2. **Дифференцирование**: затем мы дифференцируем $ \ln(L(p | X)) $ по $ p $:

   $$
   \dfrac{d \ln(L(p | X))}{dp} = \dfrac{x}{p} - \dfrac{n - x}{1 - p}
   $$

   Заметим, что логарифмирование позволило нам избавиться от константы $\ln C_n^x$.

3. **Приравнивание производной к нулю**: чтобы найти максимум, приравниваем производную к нулю и решаем относительно $ p $:

   $$
   \frac{x}{p} - \frac{n - x}{1 - p} = 0
   $$

   $$
   \Rightarrow x(1 - p) = p(n - x)
   $$

   $$
   \Rightarrow xp - xp^2 = np - xp
   $$

   $$
   \Rightarrow xp^2 - (x + n)p + x = 0
   $$

4. **Решение для $ p $**: это квадратное уравнение относительно $ p $. Оно сокращается до доли успехов, что довольно очевидно:

   $$
   \Rightarrow \hat{p}^{MLE} = \frac{x}{n}
   $$

   Таким образом, доля успехов $ \frac{x}{n} $ является MLE-оценкой для вероятности «успеха» $p$ из биномиального распределения.

#### MLE для нашей выборки

Теперь мы можем легко получить оценку $\hat{p}^{MLE}$ для нашей выборки: $ x = 7 $ и $ n = 10 $:

$$
\hat{p}^{MLE} = \frac{7}{10} = 0.7
$$

Таким образом, оценка вероятности того, что случайный индивид поддержит реформу в нашем гипотетическом референдуме, полученная с помощью MLE на основе нашей выборки, составляет 0.7.

---

Рассмотрим другой пример: нам дана выборка из биномиального распределения с параметрами $n = 10$, $p = 0.8$, состоящая из следующих наблюдений: $\{5, 7, 9\}$.

1. Чему равно правдоподобие этой выборки (какова вероятность получить эту выборку)?
2. Получим посредством метода максимального правдоподобия оценку параметра $p$ — вероятности успеха.

**Чему равно правдоподобие этой выборки (какова вероятность получить эту выборку)?**

Чтобы ответить на этот вопрос и найти $L(data | p)$, нам нужно посчитать вероятность получения каждого из элементов выборки ($ P(X = x_i) $) и перемножить их, поскольку эти события независимы. То есть:
$$
L(data | p) = P(X = 5) \cdot P(X = 7) \cdot P(X = 9)
$$

$$
P(X = 5) = \dfrac{10!}{5! \cdot 5!} \cdot 0.8^5 \cdot 0.2^5 \approx 0.026
$$
$$
P(X = 7) = \dfrac{10!}{7! \cdot 3!} \cdot 0.8^7 \cdot 0.2^3 \approx 0.201
$$
$$
P(X = 9) = \dfrac{10!}{9! \cdot 1!} \cdot 0.8^9 \cdot 0.2^1 \approx 0.268
$$

Тогда:
$$
L(data | p) = 0.026 \cdot 0.201 \cdot 0.268 \approx 0.0014
$$

Мы нашли вероятность получить выборку $\{5, 7, 9\}$ из биномиального распределения с параметрами $n = 10$, $p = 0.8$ !

**Получим посредством метода максимального правдоподобия оценку параметра $p$ — вероятности успеха.**

Выведем функцию правдоподобия:
$$
L(p | data) = ( C_{10}^5 \ p^5 (1 - p)^{5} ) \cdot ( C_{10}^7 \ p^7 (1 - p)^{3} ) \cdot ( C_{10}^9 \ p^9 (1 - p)^{1} ) = p^{21} \cdot (1 - p)^{9} \cdot C_{10}^5 \cdot C_{10}^7 \cdot C_{10}^9
$$
Прологарифмируем ее, чтобы перейти к сумме для удобства дифференцирования:
$$
\ln(L(p | data)) = \ln(p^{21}) + \ln( (1-p)^9 ) + \ln(const) + \ln(const) + \ln(const)
$$
Обозначим биномиальные коэффициенты как $const$, поскольку они не важны нам для максимизации функции.

Далее возьмем частную производную по параметру $p$ от логарифмической функции правдоподобия:
$$
\dfrac{d \ln(L(p | X))}{dp} = (21 \ln p + 9 \ln (1 - p))' = \dfrac{21}{p} + \left(-\dfrac{9}{1 - p}\right)
$$

Приравниванием производную к нулю и решим уравнение для $ p $:
$$
\dfrac{d \ln(L(p | X))}{dp} = 0
$$

$$
\Rightarrow \dfrac{21}{p} + \left(-\dfrac{9}{1 - p}\right) = 0
$$

$$
\Rightarrow \dfrac{21}{p} = \dfrac{9}{1 - p}
$$

$$
\Rightarrow 9 p = 21 (1 - p)
$$

$$
\Rightarrow 30 p = 21
$$

$$
\Rightarrow \hat{p}^{MLE} = 0.7
$$

Мы нашли MLE-оценку параметра $p$! Можно заметить, что она равна среднему арифметическому из $\{0.5, 0.7, 0.9\}$ — долей успехов из нашей выборки.

#### Практика в Python

Попробуем решить те же задачи для биномиального распределения с использованием Python. Поскольку мы будем использовать язык программирования, а не аналитическое решение, попробуем решить задачу с помощью логарифмирования функции правдоподобия и без.

**Поиск оценки параметра $p$ для выборки $X = \{1, 0, 1, 1, 0, 1, 1, 1, 0, 1\}$**

Решим задачу про референдум. Будем использовать для этого:
- объект `binom` из модуля `scipy.stats` ([документация](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.binom.html))
- и функцию `minimize_scalar` из модуля `scipy.optimize` ([документация](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize_scalar.html))

Обратите внимание, что из-за особенности реализации библиотеки `scipy` технически мы будем минимизировать функцию правдоподобия, и для этого нам придется взять ее со знаком «минус», чтобы получить правильный результат.

In [ ]:
import numpy as np
from scipy.stats import binom
from scipy.optimize import minimize_scalar

# Пример: MLE для биномиального распределения
# Выборка:
x = [1, 0, 1, 1, 0, 1, 1, 1, 0, 1]

# Число успехов
votes_received = sum(x)
# Размер выборки
total_votes = len(x)

# Функция правдоподобия для биномиального распределения
def binom_likelihood(p):
    # Обратите внимание, что мы берем отрицательное значение функции правдоподобия,
    # поскольку на следующем шаге мы будем минимизировать ее (а не максимизировать)
    return -binom.pmf(votes_received, total_votes, p)

# Найдем MLE для p
# method='bounded', поскольку мы знаем, что параметр p лежит в границах [0, 1]
result = minimize_scalar(binom_likelihood, bounds=(0, 1), method='bounded')

# Результат:
mle_p = result.x

print(f'MLE-оценка для параметра p: {mle_p} (без логарифмирования функции правдоподобия)')

MLE-оценка для параметра p: 0.699999913744543 (без логарифмирования функции правдоподобия)


Получили практически точную MLE-оценку по сравнению с аналитическим решением! Обратите внимание, что в этом случае мы не использовали логарифмирование функции правдоподобия, а брали результат `binom.pmf` напрямую (`binom.pmf` = $ P(X = x) $)

Теперь найдем оценку, используя логарифмирование:

In [ ]:
# Логарифмированная функция правдоподобия для биномиального распределения
def binom_log_likelihood(p):
    # Обратите внимание, что мы берем отрицательное значение функции правдоподобия,
    # поскольку на следующем шаге мы будем минимизировать ее (а не максимизировать)
    return -np.log(binom.pmf(votes_received, total_votes, p))

# Найдем MLE для p
# method='bounded', поскольку мы знаем, что параметр p лежит в границах [0, 1]
result = minimize_scalar(binom_log_likelihood, bounds=(0, 1), method='bounded')

# Результат:
mle_p = result.x

print(f'MLE-оценка для параметра p: {mle_p} (с логарифмированием функции правдоподобия)')

MLE-оценка для параметра p: 0.7000003717368305 (с логарифмированием функции правдоподобия)


Погрешность немного больше, но, в целом, результат тоже близкий к аналитическому решению ($0.7$).

**Правдоподобие выборки $X = \{5, 7, 9\}$ и MLE-оценка параметра $p$**

Далее решим вторую задачу, в которой нужно было найти правдоподобие выборки и получить оценку параметра $p$ для выборки из трех значений.

Здесь будем использовать только вариант с логарифмированием и попробеум записать негативность лог-правдоподобия через другой синтаксис.

In [ ]:
# Выборка из биномиального распределения
sample = [5, 7, 9]
# Число экспериментов
n = 10
# Вероятность успеха
p_given = 0.8

# Task 1: вычисление правдоподобия (likelihood) выборки
# Правдоподобие - произведение вероятностей для каждого элемента выборки
likelihood = np.prod(binom.pmf(sample, n, p_given))  # np.prod перемножает все элементы массива

# Task 2: найдем MLE-оценку для параметра p
# Определим функцию логарифмического правдоподобия
def log_likelihood(p):
    return np.sum(np.log(binom.pmf(sample, n, p)))  # помним про суммирование вместо перемножения

# Найдием значение оценки параметра p, которое максимизирует логарифмическое правдоподобие
# Используем lambda-функцию, чтобы передать в minimize_scalar отрицательное значение log_likelihood
mle_result = minimize_scalar(lambda p: -log_likelihood(p), bounds=(0, 1), method='bounded')

# Результат:
mle_p = mle_result.x

print(f'Правдоподобие выборки X = [5, 7, 9]: {likelihood}')
print()
print(f'MLE-оценка для параметра p: {mle_p} (с логарифмированием функции правдоподобия)')

Правдоподобие выборки X = [5, 7, 9]: 0.0014280436244197769

MLE-оценка для параметра p: 0.7000003717368987 (с логарифмированием функции правдоподобия)


Получили правильный ответ! Запустив функцию `binom.pmf(sample, n, p)` для каждого из элемента выборки, можно посмотреть на конкретные значения вероятности для каждого элемента, как мы считали вручную:

In [ ]:
for x_i in sample:
    print(f'P(X = {x_i}) = {binom.pmf(x_i, 10, 0.8)}')

P(X = 5) = 0.026424115199999963
P(X = 7) = 0.20132659199999978
P(X = 9) = 0.26843545599999996


### 3. Пример MLE для распределения Пуассона <a name="par3"></a>

#### Вспоминаем распределение Пуассона

Для начала вспомним суть **распределения Пуассона**.

Распределение Пуассона — это дискретное вероятностное распределение, которое выражает вероятность наступления определенного количества событий (поэтому дискретное), происходящих в фиксированный интервал времени или пространства, при условии, что эти события происходят с известной постоянной средней скоростью и независимо друг от друга (т.е. от времени с момента последнего события).

Параметр этого распределения: $\lambda > 0$, средняя скорость, с которой происходят события (или $ n \cdot p $, где $ n $ велико, а $ p $ — мало). Предполагается, что эта скорость постоянна.

В целом же, **функция вероятности** для него выглядит так:
$$
P(X = x) = \dfrac{\lambda ^x}{x!} \cdot e^{- \lambda}
$$,

где $x$ — число событий, наступивших за период времени.

Обычно распределение Пуассона используется для моделирования редких событий, где количество случаев за период времени невелико, потому что при больших значениях $\lambda > 10$ распределение Пуассона аппроксимируется к нормальному распределению.

К примеру, в политической науке его можно было бы использовать для оценки количества избирателей, приходящих на избирательный участок в течение заданного периода времени или для анализа упоминаний политиков в СМИ за определенный период.

#### Нахождение оценки MLE для распределения Пуассона

Попробуем вывести оценку параметра $\lambda $ и решить следующую задачу:

##### Задача

В городе N проходят парламентские выборы.
Для эффективной организации работы наблюдателей нам нужно понять частоту, с которой избиратели приходят на участок за один час, поскольку оплата за работу наблюдателей у нас почасовая.

Нам нужно найти оценку среднего количества избирателей, посещающих избирательный участок в течение одного часа.
Наблюдения проводились на одном из избирательных участков в городе N. Количество избирателей, приходящих за каждый час, записывалось за 12 часов работы избирательного участка.

В результате мы получили следующую выборку:

$$
X = \{2, 3, 2, 4, 1, 3, 3, 2, 4, 5, 1, 2\}
$$

Каждый элемент выборки представляет количество избирателей, приходящих в каждый из 12 часов.

Используя эти данные, найдите оценку среднего количества избирателей, приходящих на избирательный участок в час, с помощью MLE, используя распределение Пуассона.

#### Решение

Выведем оценку параметра $\lambda $ распределения Пуассона с помощью MLE в общем виде.

Мы помним как выглядит функция вероятности:
$$
P(X = x) = \dfrac{\lambda ^x}{x!} \cdot e^{- \lambda}
$$

**Функция правдоподобия**

Для выборки $X$ функция правдоподобия распределения Пуассона:

$$
L(\lambda | data) = ... \cdot \dfrac{\lambda^x}{x!} e^{- \lambda} \cdot ... = e^{-n \lambda} \cdot \lambda ^{\sum x_i} \cdot \dfrac{1}{\prod_{i=1}^{n} x_i !}
$$

Проанализируем, откуда мы взяли эти три множителя:
1. $ e^{-n \lambda} $: множитель $ \ e^{- \lambda}$ повторяется $n$ раз по числу элементов в выборке, поскольку $\lambda$ — постоянная величина, а при умножении степени складываются.
    * Например, $ e^{- \lambda} \cdot e^{- \lambda} \cdot e^{- \lambda} = e^{- 3\lambda}$
2. $ \lambda ^{\sum x_i} $: множитель в виде числителя из дроби $ \dfrac{\lambda^x}{x!} $ также повторяется $n$ раз, однако каждый раз на месте $ \lambda^x $ будет $\lambda^{i\text{-й элемент выборки}}$. Следовательно, поскольку $\lambda$ — постоянная величина, можем просуммировать элементы нашей выборки.
    * Например, для выборки $\{3, 5, 7 \}$ получим $ \lambda^3 \cdot \lambda^5 \cdot \lambda^7 = \lambda^{15} $
3. $ \dfrac{1}{\prod_{i=1}^{n} x_i !} $: все просто — каждый раз подставляем в знаменатель из дроби $ \dfrac{\lambda^x}{x!} $ новый элемент выборки $x_i$, в результате получаем произведение из факториалов всех элементов нашей выборки (напомним, что символ $\prod_{i=1}^{n}$ означает произведение элементов по аналогии с символом суммирования $\sum_{i=1}^{n}$).
    * Например, для выборки $\{1, 2, 3 \}$ получим $ 1! \cdot 2! \cdot 3! = 1 \cdot 2 \cdot 6 = 12$
        * Напомним, что факториал числа $x!$ равен $ x \cdot (x - 1) \cdot ... \cdot 1 $ или $ \prod_{k=1}^{x} k $  

**Логарифмическая функция правдоподобия**

Далее прологарифмируем функцию правдоподобия

$$
\ln(L(\lambda | data)) = -n \cdot \lambda + \sum x_i \cdot \ln(\lambda) + \ln \left(\prod_{i=1}^{n} x_i ! \right)
$$

Помним, что последнее слагаемое для нас будет являться константой при дифференцировании.

**Дифференцирование и максимизация**

Как и до этого, для нахождения оценки MLE, мы дифференцируем логарифмическую функцию правдоподобия по параметру и приравниваем ее к нулю:

$$
\dfrac{d \ln(L(\lambda | data))}{d \lambda} = -n + \sum x_i \cdot \dfrac{1}{\lambda}
$$

Приравниваем к 0:

$$
\dfrac{d \ln(L(\lambda | data))}{d \lambda} = 0
$$

$$
\Rightarrow -n + \sum x_i \cdot \dfrac{1}{\lambda} = 0
$$
$$
\Rightarrow \hat{\lambda}^{MLE} = \dfrac{\sum x_i}{n} = \bar{x}
$$

Пришли к логичному выводу, что MLE оценка параметра $ \lambda $ для распределения Пуассона равна среднему арифметическому числа событий.

Теперь мы можем легко найти ответ:

$$
\hat{\lambda}^{MLE} = \bar{x} = \dfrac{2 + 3 + 2 + 4 + 1 + 3 + 3 + 2 + 4 + 5 + 1 + 2}{12} = 2.(6)
$$

Значит оценка среднего числа избирателей, которое приходит на участок за 1 час примерно равно $2.67$

$$
L(\lambda | data) = ... \cdot \dfrac{\lambda^x}{x!} e^{- \lambda} \cdot ... = e^{-n \lambda} \cdot \lambda ^{\sum x_i} \cdot \dfrac{1}{\prod_{i=1}^{n} x_i !}
$$

Проанализируем, откуда мы взяли эти три множителя:
1. $ e^{-n \lambda} $: множитель $ \ e^{- \lambda}$ повторяется $n$ раз по числу элементов в выборке, поскольку $\lambda$ — постоянная величина, а при умножении степени складываются.
    * Например, $ e^{- \lambda} \cdot e^{- \lambda} \cdot e^{- \lambda} = e^{- 3\lambda}$
2. $ \lambda ^{\sum x_i} $: множитель в виде числителя из дроби $ \dfrac{\lambda^x}{x!} $ также повторяется $n$ раз, однако каждый раз на месте $ \lambda^x $ будет $\lambda^{i\text{-й элемент выборки}}$. Следовательно, поскольку $\lambda$ — постоянная величина, можем просуммировать элементы нашей выборки.
    * Например, для выборки $\{3, 5, 7 \}$ получим $ \lambda^3 \cdot \lambda^5 \cdot \lambda^7 = \lambda^{15} $
3. $ \dfrac{1}{\prod_{i=1}^{n} x_i !} $: все просто — каждый раз подставляем в знаменатель из дроби $ \dfrac{\lambda^x}{x!} $ новый элемент выборки $x_i$, в результате получаем произведение из факториалов всех элементов нашей выборки (напомним, что символ $\prod_{i=1}^{n}$ означает произведение элементов по аналогии с символом суммирования $\sum_{i=1}^{n}$).
    * Например, для выборки $\{1, 2, 3 \}$ получим $ 1! \cdot 2! \cdot 3! = 1 \cdot 2 \cdot 6 = 6$
        * Напомним, что факториал числа $x!$ равен $ x \cdot (x - 1) \cdot ... \cdot 1 $ или $ \prod_{k=1}^{x} k $  

#### Решение в Python

Как и раньше, попробуем «вручную» оптимизировать функцию правдоподобия для решения этой задачи. Воспользуемся уже знакомым синтаксисом и функцией `minimize_scalar`.

In [ ]:
# документация: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.poisson.html

import numpy as np
from scipy.stats import poisson
from scipy.optimize import minimize_scalar

# Выборка
rally_counts = [2, 3, 2, 4, 1, 3, 3, 2, 4, 5, 1, 2]

# Функция логарифмического правдоподобия для распределения Пуассона
def log_likelihood(lambda_param):
    # logpmf: logarithm of the probability mass function
    return np.sum(poisson.logpmf(rally_counts, lambda_param))

# Нахождение lambda, которая максимизирует логарифмическое правдоподобие
result = minimize_scalar(lambda lambda_param: -log_likelihood(lambda_param), bounds=(0, 10), method='bounded')

# MLE для lambda
mle_lambda = result.x

print(f'Выборка: {rally_counts}')
print(f'MLE-оценка для параметра lambda: {mle_lambda} (с логарифмированием функции правдоподобия)')

Выборка: [2, 3, 2, 4, 1, 3, 3, 2, 4, 5, 1, 2]
MLE-оценка для параметра lambda: 2.6666681102332452 (с логарифмированием функции правдоподобия)


Получили очень точный результат!

Обратите внимание, что мы использовали функцию `poisson.logpmf` из библиотеки `scipy`, которая, в отличие от обычной обычной `pmf`, сразу возвращает логарифмированный результат, удобный для нас. Можете сами попробовать изменить `poisson.logpmf` на `poisson.pmf`, а `np.sum` на `np.prod` и сравнить результат.

### 4. Пример MLE для экспоненциального распределения <a name="par4"></a>

#### Вспоминаем экспоненциальное распределение

Вспомним основные характеристики экспоненциального распределения.

Экспоненциальное распределение — непрерывное распределение вероятности, моделирующее время между двумя последовательными свершениями одного и того же события.

В отличие от *дискретных* распределений, которые используются для счетных событий (например, распределение Пуассона), *непрерывное* распределение, подобное экспоненциальному, используется для исследования непрерывных величин, например, временных интервалов.

Как обсуждалось ранее, **экспоненциальное распределение** широко используется в социальных и естественных науках для моделирования времени до наступления события.

**Функция плотности** экспоненциального распределения для произвольной величины выглядит следующим образом:
$$
\lambda e^{-\lambda x},
$$
где
$\lambda = \dfrac{1}{\theta} $ и $\lambda > 0$ — параметр «интенсивности» события или частота, а $\theta$ — средняя продолжительность; при этом $ x \geq 0 $.

#### Нахождение оценки MLE для экспоненциального распределения

Разовьем идею из задачи для распределения Пуассона, которую мы решали ранее — тогда мы анализировали частоту явки избирателей на парламентских выборах в городе N.

Посмотрим на продолжительность времени, которое избиратели проводят в кабинке для голосования, принимая свое решение (и, возможно, зайдем в область политической психологии). Это классический сценарий для применения экспоненциального распределения, поскольку он включает оценку средней продолжительности непрерывного процесса (времени, проведенного избирателями в кабинке).

Найдем оценку среднего времени, которое избиратель проводит в кабинке для голосования, с помощью MLE.

Предположим, что наблюдения проводились на том же избирательном участке. Записывалось время (в минутах), которое каждый избиратель проводил в кабинке для голосования. Мы имеем следующую выборку из 10 человек:
$$
X = \{3, 5, 2, 4, 3, 5, 4, 6, 2, 5\}
$$

В начале выведем оценку $\theta$, т.е. средней продолжительности, в общем виде:

**Функция правдоподобия**
$$
L(\theta | data) = ... \cdot \dfrac{1}{\theta} \cdot e^{- \frac{x}{\theta} } \cdot ... = \left( \dfrac{1}{\theta} \right)^n \cdot e^{- \frac{1}{\theta} \cdot \left( \sum x_i \right) }
$$
**Логарифмическая функция правдоподобия**
$$
\ln(L(\theta | data)) = -n \ln(\theta) - \dfrac{1}{\theta} \cdot \sum x_i
$$
**Дифференцирование и максимизация**
$$
\dfrac{d \ln(L(\theta | data))}{d \theta} = - \dfrac{n}{\theta} - \sum x_i \cdot \cdot \left( - \dfrac{1}{\theta^2} \right)
$$
$$
\dfrac{d \ln(L(\theta | data))}{d \theta} = 0
$$
$$
\Rightarrow - \dfrac{n}{\theta} - \sum x_i \cdot \left( - \dfrac{1}{\theta^2} \right) = 0
$$
$$
\Rightarrow \hat{\theta}^{MLE} = \dfrac{\sum x_i}{n} = \bar{x}
$$

Теперь мы можем легко найти ответ на задачу:

$$
\hat{\theta}^{MLE} = \bar{x} = \dfrac{3 + 5 + 2 + 4 + 3 + 5 + 4 + 6 + 2 + 5}{10} = 3.9
$$

Выходит, что, согласно нашим данным, в среднем, избиратель проводит $3.9$ минут в избирательной кабинке на выборах в городе N.

**Усложним условие**: пускай в функции правдоподобия будут участвовать как функции распределения, так и функции надежности. Например, скажем, что мы точно знаем время для 6 избирателей: 3, 2, 4, 3, 4, 2, а для четырех — время голосования составило 5 минут и более (раньше мы считали, что трое из этих избирателей сделали свой выбор за 5 минут, а один — за 6).

Тогда наша выборка будет выглядеть следующим образом:
|    t     |     n    |
|----------|----------|
| 2 min    |      2         |
| 3 min    |      2         |
| 4 min    |      2         |
| [5 min; $+ \infty$) | 4   |

В такой ситуации расчет функции правдоподобия немного изменится: мы будем комбинировать функции распределения и функции надежности, которые мы уже обсуждали подробно в контексте экспоненциального распределения. Посчитаем MLE-оценку вручную:



**Функция правдоподобия**
$$
L(\theta | data) =  \left( \dfrac{1}{\theta} \cdot e^{- \frac{2}{\theta}} \right)^2 \cdot \left( \dfrac{1}{\theta} \cdot e^{- \frac{3}{\theta}} \right)^2 \cdot
\left( \dfrac{1}{\theta} \cdot e^{- \frac{4}{\theta}} \right)^2 \cdot \left( e^{- \frac{5}{\theta}} \right)^4 = \left( \dfrac{1}{\theta}  \right)^6 \cdot
e^{- \frac{38}{\theta}}
$$
**Логарифмическая функция правдоподобия**
$$
\ln(L(\theta | data)) = - 6 \ln(\theta) - \dfrac{38}{\theta}
$$
**Дифференцирование и максимизация**
$$
\dfrac{d \ln(L(\theta | data))}{d \theta} = - \dfrac{6}{\theta} + \dfrac{38}{\theta^2}
$$
$$
- \dfrac{6}{\theta} + \dfrac{38}{\theta^2} = 0
$$
$$
\Rightarrow \dfrac{6}{\theta} = \dfrac{38}{\theta^2}
$$
$$
\Rightarrow \hat{\theta}^{MLE} = \dfrac{38}{6} = 6.(3)
$$

Получили, что с измененным условием, когда наблюдения $5, 5, 5, 6$ «превратились» в четыре $5+$, оценка математического ожидания среднего времени голосования увеличилась с $3.9$ до $6.(3)$

#### Решение в Python

Попробуем решить обе версии задачи с помощью Python: как только с функциями распределения, так и дополнительно с функциями надежности.

Для расчета логарифмической функции правдоподобия будем использовать две функции из `scipy.stats`:
- `expon.logpdf`: log of the probability density function
- `expon.logsf`: log of the survival function

В начале решим первую вариацию задания:

In [ ]:
# Документация: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.expon.html

import numpy as np
from scipy.stats import expon
from scipy.optimize import minimize_scalar

# Выборка
voting_time = [3, 5, 2, 4, 3, 5, 4, 6, 2, 5]

# Функция логарифмического правдоподобия для экспоненциального распределения
def exp_log_likelihood(lambda_param):
    # Технически удобнее оптимизировать lambda, а потом выразить из нее theta
    pdf_contrib = np.sum(expon.logpdf(voting_time, scale=1/lambda_param))

    return pdf_contrib

# Нахождение lambda, которая максимизирует логарифмическое правдоподобие
result = minimize_scalar(lambda x: -exp_log_likelihood(x), bounds=(0, 1), method='bounded')

# MLE для lambda
mle_lambda = result.x

print(f'Выборка: {voting_time}')
print(f'MLE-оценка для параметра theta: {1 / mle_lambda} (с логарифмированием функции правдоподобия)')

Выборка: [3, 5, 2, 4, 3, 5, 4, 6, 2, 5]
MLE-оценка для параметра theta: 3.900013550547837 (с логарифмированием функции правдоподобия)


Теперь решим усложненный вариант задания. Поскольку мы используем логарифм правдоподобия, можем по отдельности найти значения функций распределения и функций надежности, а затем сложить их:

In [ ]:
# Выборка
voting_time = [3, 2, 4, 3, 4, 2]  # остальные значения передадим напрямую

# Функция логарифмического правдоподобия для экспоненциального распределения
def extended_exp_log_likelihood(lambda_param):
    # Технически удобнее оптимизировать lambda, а потом выразить из нее theta
    pdf_contrib = np.sum(expon.logpdf(voting_time, scale=1/lambda_param))

    # Считаем логарифм функции надежности
    survival_contrib = np.sum(expon.logsf([5] * 4, scale=1/lambda_param))

    return pdf_contrib + survival_contrib

# Нахождение lambda, которая максимизирует логарифмическое правдоподобие
result = minimize_scalar(lambda x: -extended_exp_log_likelihood(x), bounds=(0, 1), method='bounded')

# MLE для lambda
mle_lambda = result.x

print(f"Выборка: {', '.join(list(map(str, voting_time)) + ['5+', '5+', '5+', '5+'])}")
print(f'MLE-оценка для параметра theta: {1 / mle_lambda} (с логарифмированием функции правдоподобия)')

Выборка: 3, 2, 4, 3, 4, 2, 5+, 5+, 5+, 5+
MLE-оценка для параметра theta: 6.333278679750698 (с логарифмированием функции правдоподобия)


Оба раза получили правильный ответ!

### 5. Пример MLE для нормального распределения <a name="par5"></a>

#### Вспоминаем нормальное распределение

Как мы помним, нормальное распределение очень важно для теории вероятностей, оно широко используется в различных прикладных областях и в самой теории вероятностей и математической статистике.

В политической науке нормальное распределение может использоваться для анализа поведения избирателей, опросов общественного мнения, экономических показателей, и многого другого. Также оно часто используется при проверке гипотез и оценке доверительных интервалов, а также в качестве основы для более сложных статистических моделей.

#### Функция плотности вероятности

Функция плотности вероятности нормального распределения задается формулой:

$$
f(x) = \frac{1}{\sqrt{2\pi}\sigma} e^{-\frac{(x - \mu)^2}{2\sigma^2}}
$$,
где $ \sigma $ — стандартное отклонение, и $ \mu $ — среднее значение распределения.

Рассмотрим стандартное нормальное распределение и будем в дальнейшем работать с ним для простоты.
Напомним, что у стандартного нормального распределения $ \sigma = 1 $ и $ \mu = 0$.

#### Нахождение оценки MLE для нормального распределения (для среднего значения)

Попробуем вывести оценку параметра $\mu $ и решить следующую задачу:

##### Задача

Представим, что социологическое агентство обратилось к нам с просьбой проанализировать опрос общественного мнения о поддержке кандидата N.
Мы знаем, что выборка пришла к нам из нормального распределения со стандартным отклонением, равным 1.

Мы имеем следующую выборку (шкала от 1 до 10, где 1 означает решительное несогласие, а 10 — решительную поддержку):
$$
X = \{6, 8, 7, 5, 9, 7, 6, 8, 5, 7\}
$$

Нам необходимо найти оценку среднего значения поддержки политика N с помощью MLE.

##### Решение

**Функция правдоподобия**: воспользуемся тем, что $ \sigma = 1 $
$$
L(\mu | data) = ... \cdot  \frac{1}{\sqrt{2\pi}} e^{-\frac{(x - \mu)^2}{2}} \cdot ... = \left( \dfrac{1}{\sqrt{2\pi}} \right)^n \cdot e^{- \frac{1}{2} \sum_{i=1}^n (x_i - \mu)^2}
$$
**Логарифмическая функция правдоподобия**
$$
\ln(L(\mu | data)) = \ln\left( \left( \dfrac{1}{\sqrt{2\pi}} \right)^n \right) - \frac{1}{2} \sum_{i=1}^n (x_i - \mu)^2
$$

Как можно понять, $ \ln\left( \left( \dfrac{1}{\sqrt{2\pi}} \right)^n \right) $ будет являться для нас константой при дифференцировании.

**Дифференцирование и максимизация**
$$
\dfrac{d \ln(L(\mu | data))}{d \mu} = - \frac{1}{2} \sum_{i=1}^n (- 2 (x_i - \mu) )
$$
$$
- \frac{1}{2} \sum_{i=1}^n (- 2 (x_i - \mu) ) = 0
$$
$$
\Rightarrow \sum_{i=1}^n (x_i - \mu) = 0
$$
$$
\Rightarrow \sum_{i=1}^n x_i - n \mu = 0
$$
$$
\Rightarrow \hat{\mu}^{MLE} = \dfrac{\sum_{i=1}^n x_i }{n} = \bar{x}
$$

Мы получили, что MLE-оценкой параметра $ \mu $ нормального распределения (при условии $ \sigma = 1 $) является выборочное среднее! На самом деле, если опустить условие $ \sigma = 1 $, результат будет такой же, но вывод несколько усложнится.



Стоит отметить *свойства* этой оценки:
- **несмещенная**: математическое ожидание оценки равно оцениваемому параметру или $E(\hat{\theta}) = \theta$
- **состоятельная**: сходится по вероятности к истинному значению параметра по мере увеличения размера выборки (с увеличением объема выборки оценка становится более точной)
или, если для любого $ \epsilon > 0: \ \lim_{n \to \infty} P(|\hat{\theta}_n - \theta| < \epsilon) = 1 $
- **эффективная**: среди всех несмещенных оценок, эффективная оценка имеет наименьшую дисперсию (оценка использует выборочные данные наиболее эффективно) или
между двумя несмещенными оценками $ \hat{\theta}_1 $ и $ \hat{\theta}_2 $, $ \hat{\theta}_1 $ более эффективна, чем $ \hat{\theta}_2 $, если $Var(\hat{\theta}_1) < Var(\hat{\theta}_2) $
- **НЕустойчивая**: чувствительна к выбросам в данных (в отличие от медианы, например)


Теперь мы можем легко решить задачу!

$$
\hat{\mu}^{MLE} = \bar{x} = \dfrac{6 + 8 + 7 + 5 + 9 + 7 + 6 + 8 + 5 + 7}{10} = 6.8
$$

Получается, что MLE-оценка среднего значения поддержки кандидата N равна $ 6.8 $.

#### Решение в Python

Для расчета логарифмической функции правдоподобия будем использовать функцию из `scipy.stats`: `norm.logpdf` (log of the probability density function).

In [ ]:
# Документация: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html

import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize_scalar

# Выборка
opinion_ratings = [6, 8, 7, 5, 9, 7, 6, 8, 5, 7]

# Функция логарифмического правдоподобия для экспоненциального распределения
def normal_log_likelihood(mu):
    # Берем с минусом, потому что будем минимизировать
    return -np.sum(norm.logpdf(opinion_ratings, loc=mu, scale=1))  # предположение: sigma = 1

# Нахождение mu, которая максимизирует логарифмическое правдоподобие
normal_result = minimize_scalar(normal_log_likelihood)

# MLE для mu
mle_mu = normal_result.x

print(f"Выборка: {opinion_ratings}")
print(f'MLE-оценка для параметра mu: {mle_mu} (с логарифмированием функции правдоподобия)')

Выборка: [6, 8, 7, 5, 9, 7, 6, 8, 5, 7]
MLE-оценка для параметра mu: 6.80000000000001 (с логарифмированием функции правдоподобия)


Получили правильный результат!